In [1]:
from flask import Flask, request, jsonify
import os
import sys

sys.path.append("./api")
from api.scripts.visualize_box_version import (
    prepare_dataset_and_model,
    generate_queried_unit_mesh,
)

args_location = "api/test/new_prior_1500/args.json"
ckpt_link = (
    "https://drive.google.com/file/d/1-YjqqoAnv9_FD288mRYIGoNW75CcJaUR/view?usp=sharing"
)

In [2]:
args, model, dataset, _, _ = prepare_dataset_and_model(
    args_location=args_location, ckpt_epoch=400, ckpt_link=ckpt_link
)

e:\neuralroom_nextjs\api\model\VAE_prior.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


model loaded!
training statistics collected


In [6]:
from api.LLM.prompt_basic import make_basic_input_prompt, make_basic_sysprompt
from openai import OpenAI

client = OpenAI()
description = "A layout designed for a single professional who works from home and enjoys painting. They need a dedicated workspace and an art studio. "
sysprompt = make_basic_sysprompt()
input = make_basic_input_prompt(description)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": sysprompt},
        {"role": "user", "content": input},
    ],
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Step 1:  Determine Headcounts and Basic Room Requirements\n-Occupant: 1 person.\n-Bedrooms: 1 bedroom.\n-Bathrooms: 1 bathroom.\n-Basic room list: [\'bedroom\', \'bathroom\']\n\nStep 2:  Additional rooms based on lifestyle\n-Works from home: include a \'library\' for workspace.\n-Enjoys painting: since \'artstudio\' is not in the predefined list, use \'library\' also to serve as an art space.\n-Common areas: \'livingroom\', \'kitchen\'.\n-Complete room list: [\'bedroom\', \'bathroom\', \'livingroom\', \'kitchen\', \'library\']\n\nStep 3: Assign Indexes\n{\n  \'0\': \'bedroom\',\n  \'1\': \'bathroom\',\n  \'2\': \'livingroom\',\n  \'3\': \'kitchen\',\n  \'4\': \'library\'\n}\n\nStep 4: Infer Adjacency\n-Bedroom connected to bathroom: (0, 1).\n-Bedroom connected to library (workspace and art space): (0, 4).\n-Living room connected to kitchen: (2, 3).\n-Living room connected to library: (2, 4).\n-Adjacency List: [(0, 1), (0, 4), (2, 3), (2, 4)]\n\nStep 5:\n{

In [5]:
from pydantic import BaseModel
from typing import List, Tuple, Dict


from openai import OpenAI


client = OpenAI()


class RoomAdjacency(BaseModel):
    room1: int
    room2: int


class Room(BaseModel):
    index: int
    room_type: str


class RoomGraph(BaseModel):
    roomlist: List[Room]
    adjacencylist: List[RoomAdjacency]


class OutputGraph(BaseModel):
    step1: str
    step2: str
    step3: str
    step4: str
    step5: RoomGraph


client = OpenAI()
description = "help me design a layout for a family of 4 -- one couple and two children. The couple shares a bedroom, and the each child has own bedroom"
sysprompt = make_basic_sysprompt()
input = make_basic_input_prompt(description)


completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": sysprompt},
        {"role": "user", "content": input},
    ],
    response_format=OutputGraph,
)


event = completion.choices[0].message.parsed
print(event)

step1="Occupants: 4 people (parents and two children). Bedrooms: Parents share one bedroom; each child has their own bedroom. Total bedrooms = 3. Bathrooms: For 4 people, 1 bathroom is suitable but having 2 bathrooms could provide additional convenience. Basic room list: ['bedroom', 'bedroom', 'bedroom', 'bathroom', 'bathroom']" step2="Common areas: 'livingroom', 'kitchen', 'diningroom'. Circulation space: 'circulation'. Complete room list: ['bedroom', 'bedroom', 'bedroom', 'bathroom', 'bathroom', 'livingroom', 'kitchen', 'diningroom', 'circulation']" step3="Assign Indexes {'0': 'bedroom', '1': 'bedroom', '2': 'bedroom', '3': 'bathroom', '4': 'bathroom', '5': 'livingroom', '6': 'kitchen', '7': 'diningroom', '8': 'circulation'}" step4='Bedrooms connected to bathrooms: (0, 3), (1, 4), (2, 4). Bedrooms connected to circulation: (0, 8), (1, 8), (2, 8). Circulation connected to living room: (8, 5). Living room connected to kitchen and dining room: (5, 6), (5, 7). Kitchen connected to dining

In [9]:
gt_dir = "api/test/new_prior_1500_GT"
class_file = gt_dir + "/classes.txt"


def process_LLM_output(classes_dict, rel_dict, LLM_output=None):
    """
    input
    - class_dic: the obj classes dict from the dataset
    - rel_dic: the relationship dict from the dataset
    - LLM_output: the raw output of LLM, including step1,step2,step3

    return
    - room_list: a list of room class integer id
    - adj_list: a list of room adjcency tuple, (room1 index, room2 index)

    """

    room_graph = LLM_output.step5
    room_str_list = [room.room_type for room in room_graph.roomlist]
    room_list = [classes_dict[rm] for rm in room_str_list]
    adj_id = rel_dict.get("adjacent to")
    adj_list = [[adj.room1, adj_id, adj.room2] for adj in room_graph.adjacencylist]
    return room_list, adj_list


classes_dict = dataset.classes
rel_dict = dataset.relationships_dict
room_list, adj_list = process_LLM_output(
    classes_dict=classes_dict, rel_dict=rel_dict, LLM_output=event
)

print(room_list)
print(adj_list)

[3, 3, 3, 1, 1, 13, 11, 9, 6]
[[0, 3, 3], [1, 3, 4], [2, 3, 4], [0, 3, 8], [1, 3, 8], [2, 3, 8], [8, 3, 5], [5, 3, 6], [5, 3, 7], [6, 3, 7]]


In [4]:
import os
import sys

sys.path.append("api/")

from api.scripts.visualize_box_version import (
    prepare_dataset_and_model,
    generate_queried_unit_mesh,
)


gt_dir = "api/test/new_prior_1500_GT"
args_location = "api/test/new_prior_1500/args.json"
ckpt_link = (
    "https://drive.google.com/file/d/1-YjqqoAnv9_FD288mRYIGoNW75CcJaUR/view?usp=sharing"
)
# model and dataset initialized when webpage mounted
args, model, dataset, _, _ = prepare_dataset_and_model(
    args_location=args_location, ckpt_epoch=400, ckpt_link=ckpt_link
)
print("model initialized")

e:\neuralroom_nextjs\api\model\VAE_prior.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


model loaded!
training statistics collected
model initialized


In [5]:
print(dataset.classes)

{'_unit_': 0, 'bathroom': 1, 'bed': 2, 'bedroom': 3, 'cabinet': 4, 'chair': 5, 'circulation': 6, 'courtyard': 7, 'desk': 8, 'diningroom': 9, 'empty': 10, 'kitchen': 11, 'library': 12, 'livingroom': 13, 'nightstand': 14, 'service': 15, 'sofa': 16, 'storage': 17, 'table': 18, 'wardrobe': 19}


In [6]:
model_file_path = generate_queried_unit_mesh(
    input_objs=room_list,
    input_triples=adj_list,
    unit_box=[3, 3, 5],
    args=args,
    model=model,
    train_dataset=dataset,
)

IndexError: index out of range in self